In [ ]:
!pip install kashgari==1.1.5
# !pip install "kashgari>=1.1,<2.0"
!pip install "tensorflow>=1.14,<2.0"
!pip install tensorflow-gpu==1.15.0
!pip install keras-applications==1.0.8
!pip install keras==2.3.1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Read Data

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
 
import os
import pandas as pd
import numpy as np

os.chdir(r"/content/drive/My Drive/Colab Notebooks/2020_玉山夏季賽_NLP應用挑戰賽/t-brain_2020_NLP_team_share_folder")
# os.chdir(r"/content/drive/My Drive/Colab Notebooks/t-brain_2020_NLP_team_share_folder")


import pickle
with open('raw_data/all_content_emptyname_maxLen_0628.pkl', 'rb') as f:  # all_content_emptyname_maxLen_0802
  all_content = pickle.load(f)
with open('raw_data/all_BIO_emptyname_maxLen_0628.pkl', 'rb') as f: # all_BIO_emptyname_maxLen_0802
  all_BIO = pickle.load(f)
with open('raw_data/all_idx_emptyname_maxLen_0628.pkl', 'rb') as f: # all_idx_emptyname_maxLen_0802
  all_page_idx = pickle.load(f)

dataset = pd.DataFrame({'content':all_content, 'BIO':all_BIO, 'page_idx':[i[0] for i in all_page_idx], 'idx':all_page_idx})
dataset["NameEmpty"] = dataset["BIO"].apply(lambda x: int(all(np.array(x) == "O")))
dataset.head()

,content,BIO,page_idx,idx,NameEmpty
0,"[近, 年, 來, 投, 資, 市, 場, 波, 動, 越, 來, 越, 明, 顯, ,, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
1,"[專, 家, 表, 示, ,, 採, 用, 量, 化, 交, 易, 策, 略, 投, 資, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
2,"[大, 數, 據, 時, 代, 來, 臨, ,, 風, 行, 歐, 美, 5, 0, 年, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
3,"[舉, 辦, 「, 時, 間, 序, 列, 與, 量, 化, 交, 易, 研, 討, 會, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
4,"[越, 來, 越, 多, 的, 基, 金, 公, 司, 重, 視, 量, 化, 交, 易, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1


## Get Train/Val/Test indices

In [ ]:
def train_validate_test_split(input_df, train_percent=.6, validate_percent=.2, seed=None):
    input_list = np.unique(input_df['page_idx']).tolist()
    np.random.seed(seed)
    perm = np.random.permutation(range(len(input_list))).tolist()
    m = len(input_list)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = list(input_list[i] for i in perm[:train_end])
    validate = list(input_list[i] for i in perm[train_end:validate_end])
    test = list(input_list[i] for i in perm[validate_end:])
        
    # -- train data --
    train_data = [dataset[dataset['page_idx'].isin(train)]['content'].tolist(), dataset[dataset['page_idx'].isin(train)]['BIO'].tolist(), dataset[dataset['page_idx'].isin(train)]['idx'].tolist()]

    # -- valid data --
    valid_data = [dataset[dataset['page_idx'].isin(validate)]['content'].tolist(), dataset[dataset['page_idx'].isin(validate)]['BIO'].tolist(), dataset[dataset['page_idx'].isin(validate)]['idx'].tolist()]

    # -- test data --
    test_data = [dataset[dataset['page_idx'].isin(test)]['content'].tolist(),  dataset[dataset['page_idx'].isin(test)]['BIO'].tolist(), dataset[dataset['page_idx'].isin(test)]['idx'].tolist()]

    return train_data, valid_data, test_data

In [ ]:
train_data, valid_data, test_data = train_validate_test_split(input_df=dataset, train_percent=.6, validate_percent=.2, seed=123)
train_x, train_y, train_idx =  train_data[0], train_data[1], train_data[2]
valid_x, valid_y, valid_idx =  valid_data[0], valid_data[1], valid_data[2]
test_x, test_y, test_idx =  test_data[0], test_data[1], test_data[2]
print(f"train data count: {len(train_x)}")
print(f"validate data count: {len(valid_x)}")
print(f"test data count: {len(test_x)}")

train data count: 66804
validate data count: 20721
test data count: 20217


### Customized Model - Double LSTM


In [ ]:
from typing import Dict, Any

from tensorflow import keras

from kashgari.tasks.labeling.base_model import BaseLabelingModel
from kashgari.layers import L
from tensorflow.python.keras.layers import Layer
from keras.optimizers import Adam, SGD


from kashgari.corpus import SMP2018ECDTCorpus
from tensorflow import keras
import kashgari
from kashgari.embeddings import BERTEmbedding
from kashgari.tasks.labeling import BiLSTM_Model, CNN_LSTM_Model

from sklearn.metrics import f1_score, recall_score, precision_score

from keras.callbacks import Callback
from keras import backend as K
from kashgari.callbacks import EvalCallBack

import logging
logging.basicConfig(level='DEBUG')

class NonMasking(Layer):
    def __init__(self, **kwargs):
        self.supports_masking = True
        super(NonMasking, self).__init__(**kwargs)
    def build(self, input_shape):
        input_shape = input_shape
    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None
    def call(self, x, mask=None):
        return x
    def get_output_shape_for(self, input_shape):
        return input_shape


class DoubleBLSTMModel(BaseLabelingModel):
    """Bidirectional LSTM Sequence Labeling Model"""

    @classmethod
    def get_default_hyper_parameters(cls) -> Dict[str, Dict[str, Any]]:
        """
        Get hyper parameters of model
        Returns:
            hyper parameters dict
        """
        return {
            'layer_blstm1': {
                'units': 128,
                'return_sequences': True
            },
            'layer_blstm2': {
                'units': 128,
                'return_sequences': True
            },
            'layer_dropout': {
                'rate': 0.4
            },
            'layer_time_distributed': {},
            'layer_activation': {
                'activation': 'softmax'
            }
        }

    def build_model_arc(self):
        """
        build model architectural
        """
        output_dim = len(self.processor.label2idx)
        config = self.hyper_parameters
        embed_model = self.embedding.embed_model

        # Define your layers
        layer_blstm1 = L.Bidirectional(L.LSTM(**config['layer_blstm1']),
                                       name='layer_blstm1')
        layer_blstm2 = L.Bidirectional(L.LSTM(**config['layer_blstm2']),
                                       name='layer_blstm2')

        layer_dropout = L.Dropout(**config['layer_dropout'],
                                  name='layer_dropout')

        layer_time_distributed = L.TimeDistributed(L.Dense(output_dim,
                                                           **config['layer_time_distributed']),
                                                   name='layer_time_distributed')
        layer_activation = L.Activation(**config['layer_activation'])

        # Define tensor flow
        tensor = layer_blstm1(embed_model.output)
        tensor = layer_blstm2(tensor)
        tensor = layer_dropout(tensor)
        tensor = layer_time_distributed(tensor)
        output_tensor = layer_activation(tensor)

        # Init model
        self.tf_model = keras.Model(embed_model.inputs, output_tensor)

tf_board_callback = keras.callbacks.TensorBoard(log_dir='model/logs', update_freq=1000)

bert_embed = BERTEmbedding('code/chinese_L-12_H-768_A-12',
                                task=kashgari.LABELING,
                                sequence_length=100)

model = DoubleBLSTMModel(bert_embed)
# This step will build token dict, label dict and model structure
model.build_model(train_x, train_y, valid_x, valid_y)
# Compile model with custom optimizer, you can also customize loss and metrics.
# optimizer = RAdam()
# model.compile_model(optimizer=optimizer, loss=categorical_focal_loss(gamma=2.0, alpha=0.25))

eval_callback = EvalCallBack(kash_model=model,
                             valid_x=valid_x,
                             valid_y=valid_y,
                             step=1)

model.compile_model()

# Train model
model.fit(train_x, train_y, valid_x, valid_y, batch_size=128, epochs=5, callbacks=[eval_callback])

### Customized Model - LSTM CNN


In [ ]:
from typing import Dict, Any

from tensorflow import keras

from kashgari.tasks.labeling.base_model import BaseLabelingModel
from kashgari.layers import L
from tensorflow.python.keras.layers import Layer
from keras.optimizers import Adam, SGD


from kashgari.corpus import SMP2018ECDTCorpus
from tensorflow import keras
import kashgari
from kashgari.embeddings import BERTEmbedding
from kashgari.tasks.labeling import BiLSTM_Model, CNN_LSTM_Model

from sklearn.metrics import f1_score, recall_score, precision_score

from keras.callbacks import Callback
from keras import backend as K
from kashgari.callbacks import EvalCallBack
from keras.callbacks import ReduceLROnPlateau

import logging
logging.basicConfig(level='DEBUG')

# class NonMasking(Layer):
#     def __init__(self, **kwargs):
#         self.supports_masking = True
#         super(NonMasking, self).__init__(**kwargs)
#     def build(self, input_shape):
#         input_shape = input_shape
#     def compute_mask(self, input, input_mask=None):
#         # do not pass the mask to the next layers
#         return None
#     def call(self, x, mask=None):
#         return x
#     def get_output_shape_for(self, input_shape):
#         return input_shape


class LSTM_CNN_Model(BaseLabelingModel):
    """Bidirectional LSTM Sequence Labeling Model"""

    @classmethod
    def get_default_hyper_parameters(cls) -> Dict[str, Dict[str, Any]]:
        """
        Get hyper parameters of model
        Returns:
            hyper parameters dict
        """
        return {
            'layer_blstm1': {
                'units': 128,
                'return_sequences': True
            },
            'layer_dropout': {
                'rate': 0.4
            },
            'layer_conv1d':{
                'kernel_size': 10,
                'padding': 'same',
            },
            'layer_time_distributed': {},
            'layer_activation': {
                'activation': 'softmax'
            }
        }

    def build_model_arc(self):
        """
        build model architectural
        """
        output_dim = len(self.processor.label2idx)
        config = self.hyper_parameters
        embed_model = self.embedding.embed_model

        # Define your layers
        layer_blstm1 = L.Bidirectional(L.LSTM(**config['layer_blstm1']),
                                       name='layer_blstm1')

        layer_dropout = L.Dropout(**config['layer_dropout'],
                                  name='layer_dropout')

        layer_conv1d = L.Conv1D(output_dim, **config['layer_conv1d'])
        layer_gmp = L.GlobalMaxPooling1D()
        layer_bn = L.BatchNormalization()
        layer_activation = L.Activation(**config['layer_activation'])

        # Define tensor flow
        tensor = layer_blstm1(embed_model.output)
        tensor = layer_dropout(tensor)
        tensor = layer_conv1d(tensor)
        # tensor = layer_gmp(tensor)
        output_tensor = layer_activation(tensor)
        # 最後維度 100 * 4

        # Init model
        self.tf_model = keras.Model(embed_model.inputs, output_tensor)

tf_board_callback = keras.callbacks.TensorBoard(log_dir='model/logs', update_freq=1000)

bert_embed = BERTEmbedding('code/chinese_L-12_H-768_A-12',
                                task=kashgari.LABELING,
                                sequence_length=100)

model = LSTM_CNN_Model(bert_embed)
# This step will build token dict, label dict and model structure
model.build_model(train_x, train_y, valid_x, valid_y)
# Compile model with custom optimizer, you can also customize loss and metrics.
# optimizer = RAdam()
# model.compile_model(optimizer=optimizer, loss=categorical_focal_loss(gamma=2.0, alpha=0.25))

eval_callback = EvalCallBack(kash_model=model,
                             valid_x=valid_x,
                             valid_y=valid_y,
                             step=1)

model.compile_model()

# Train model
model.fit(train_x, train_y, valid_x, valid_y, batch_size=128, epochs=2, callbacks=[eval_callback])

### 用數據判斷判別器+關鍵字詞表的效果

In [ ]:
import kashgari

# load binary classifier
model_binary_1 = kashgari.utils.load_model('/content/drive/My Drive/Colab Notebooks/2020_玉山夏季賽_NLP應用挑戰賽/t-brain_2020_NLP_team_share_folder/model/model_binary_1.h5')

# keyword list
keyword_list = ['吸金', '收賄', '洗錢','行賄','貪汙', '貪污','貪瀆','回扣', '賄賂', "暴利",'不法獲利', '詐欺', "索賄",
                    '詐欺前科', '詐欺取財', '詐貸', '詐領', '詐騙', '詐取', '榨取', '暴力討債', '販毒', "龐式騙局", "非法獲利", 
                    '證交法', '證券交易法', '地下匯兌', '套利', '匯兌', '內線', "捲款潛逃", "捲款", '人頭戶', '仿冒品', '侵占', 
                    '偽造', '包庇', '弊案', '恐嚇取財', '掏空',  '海洛因', '炒股', '營業祕密法', '白手套', '竊盜', '期貨交易法', 
                    '經濟犯', '老鼠會', '製毒', '買票', '贓款', '走私', '逃漏', '逃漏稅', "逃稅", '銀行法', "挪用", "弊端", 
                    "涉弊", "牟利", "浮報", "虛報", "黑金", "資恐", "恐怖主義", "廉政", "收受", "假帳", "地下錢莊"]


In [ ]:
def load_AML_model(AML_model):
    global AML_classifier
    AML_classifier = AML_model

load_AML_model(AML_model=model_binary_1)

In [ ]:
def AML_predict(article):
    tensor = AML_classifier.embedding.process_x_dataset(data_preprocess(purge_char(article)))
    probs = AML_classifier.tf_model.predict(tensor)
    # print("probs", probs)
    if np.max(probs[:, 1]) < 0.3: 
        # print("Predict by AML Classifier and returned [].")
        answer = 0
    else:
        answer = 1
    return answer

def Keyword_predict(article):
    answer = 0
    for keyword in keyword_list:
        if keyword in article:
            answer = 1
            break
    return answer, keyword

In [ ]:
# Load Data
news_urls = pd.read_csv('raw_data/train_data_custom_0628.csv')

question_list = []
answer = []

for _, news in news_urls.iterrows():
    if "省略內文" not in news["content"]:
        question_list.append(news['content'])
        answer.append(int(news['name']!="[]"))


In [ ]:
import time

pred_results = {"Article":[], "Classifier_pred": [], "Keyword_pred": []}
for i in range(len(question_list)):
    
    pred_results["Article"].append(question_list[i])
    pred_results["Classifier_pred"].append(AML_predict(question_list[i]))
    pred_results["Keyword_pred"].append(Keyword_predict(question_list[i])[0])

    print("No: ", i)

In [ ]:
pred_results["Ground_truth"] = answer
result = pd.DataFrame(pred_results)
result.to_excel("compare/AML_classifier_and_keyword_list_SCORE.xlsx", index=None)

In [ ]:
# 計算F1 Score
from sklearn.metrics import f1_score

result["Classifier_with_keyword"] = ((result["Classifier_pred"] == 1) & (result["Keyword_pred"] == 1)).astype(int)

# 
print("Classifier_with_keyword: ", f1_score(result["Ground_truth"], result["Classifier_with_keyword"]))
print("Classifier_pred: ", f1_score(result["Ground_truth"], result["Classifier_pred"]))
print("Keyword_pred: ", f1_score(result["Ground_truth"], result["Keyword_pred"]))


Classifier_with_keyword:  0.9278350515463918
Classifier_pred:  0.7288888888888888
Keyword_pred:  0.7464454976303317


### 判斷AML文章的Binary Clssifier

In [ ]:
AML_y = [1 if "B-PER" in y else 0 for y in train_y]

In [ ]:
# encoding: utf-8

# author: BrikerMan
# contact: eliyar917@gmail.com
# blog: https://eliyar.biz

# file: models.py
# time: 2019-05-20 11:13

import logging
from typing import Dict, Any

from tensorflow import keras

from kashgari.tasks.labeling.base_model import BaseLabelingModel
from kashgari.layers import L
from kashgari.layers.crf import CRF

from kashgari import custom_objects

custom_objects['CRF'] = CRF


class BiLSTM_Model(BaseLabelingModel):
    """Bidirectional LSTM Sequence Labeling Model"""

    @classmethod
    def get_default_hyper_parameters(cls) -> Dict[str, Dict[str, Any]]:
        """
        Get hyper parameters of model
        Returns:
            hyper parameters dict
        """
        return {
            'layer_blstm': {
                'units': 128,
                'return_sequences': True
            },
            'layer_dropout': {
                'rate': 0.4
            },
            'layer_time_distributed': {},
            'layer_activation': {
                'activation': 'softmax'
            }
        }

    def build_model_arc(self):
        """
        build model architectural
        """
        output_dim = len(self.processor.label2idx)
        config = self.hyper_parameters
        embed_model = self.embedding.embed_model

        layer_blstm = L.Bidirectional(L.LSTM(**config['layer_blstm']),
                                      name='layer_blstm')

        layer_dropout = L.Dropout(**config['layer_dropout'],
                                  name='layer_dropout')

        layer_time_distributed = L.TimeDistributed(L.Dense(output_dim,
                                                           **config['layer_time_distributed']),
                                                   name='layer_time_distributed')
        layer_activation = L.Activation(**config['layer_activation'])

        tensor = layer_blstm(embed_model.output)
        tensor = layer_dropout(tensor)
        tensor = layer_time_distributed(tensor)
        output_tensor = layer_activation(tensor)

        self.tf_model = keras.Model(embed_model.inputs, output_tensor)


class BiLSTM_CRF_Model(BaseLabelingModel):
    """Bidirectional LSTM CRF Sequence Labeling Model"""

    @classmethod
    def get_default_hyper_parameters(cls) -> Dict[str, Dict[str, Any]]:
        """
        Get hyper parameters of model
        Returns:
            hyper parameters dict
        """
        return {
            'layer_blstm': {
                'units': 128,
                'return_sequences': True
            },
            'layer_dense': {
                'units': 64,
                'activation': 'tanh'
            }
        }

    def build_model_arc(self):
        """
        build model architectural
        """
        output_dim = len(self.processor.label2idx)
        config = self.hyper_parameters
        embed_model = self.embedding.embed_model

        layer_blstm = L.Bidirectional(L.LSTM(**config['layer_blstm']),
                                      name='layer_blstm')

        layer_dense = L.Dense(**config['layer_dense'], name='layer_dense')
        layer_crf_dense = L.Dense(output_dim, name='layer_crf_dense')
        layer_crf = CRF(output_dim, name='layer_crf')

        tensor = layer_blstm(embed_model.output)
        tensor = layer_dense(tensor)
        tensor = layer_crf_dense(tensor)
        output_tensor = layer_crf(tensor)

        self.layer_crf = layer_crf
        self.tf_model = keras.Model(embed_model.inputs, output_tensor)

    def compile_model(self, **kwargs):
        if kwargs.get('loss') is None:
            kwargs['loss'] = self.layer_crf.loss
        if kwargs.get('metrics') is None:
            kwargs['metrics'] = [self.layer_crf.viterbi_accuracy]
        super(BiLSTM_CRF_Model, self).compile_model(**kwargs)


class BiGRU_Model(BaseLabelingModel):
    """Bidirectional GRU Sequence Labeling Model"""

    @classmethod
    def get_default_hyper_parameters(cls) -> Dict[str, Dict[str, Any]]:
        """
        Get hyper parameters of model
        Returns:
            hyper parameters dict
        """
        return {
            'layer_bgru': {
                'units': 128,
                'return_sequences': True
            },
            'layer_dropout': {
                'rate': 0.4
            },
            'layer_time_distributed': {},
            'layer_activation': {
                'activation': 'softmax'
            }
        }

    def build_model_arc(self):
        """
        build model architectural
        """
        output_dim = len(self.processor.label2idx)
        config = self.hyper_parameters
        embed_model = self.embedding.embed_model

        layer_blstm = L.Bidirectional(L.GRU(**config['layer_bgru']),
                                      name='layer_bgru')

        layer_dropout = L.Dropout(**config['layer_dropout'],
                                  name='layer_dropout')

        layer_time_distributed = L.TimeDistributed(L.Dense(output_dim,
                                                           **config['layer_time_distributed']),
                                                   name='layer_time_distributed')
        layer_activation = L.Activation(**config['layer_activation'])

        tensor = layer_blstm(embed_model.output)
        tensor = layer_dropout(tensor)
        tensor = layer_time_distributed(tensor)
        output_tensor = layer_activation(tensor)

        self.tf_model = keras.Model(embed_model.inputs, output_tensor)


class BiGRU_CRF_Model(BaseLabelingModel):
    """Bidirectional GRU CRF Sequence Labeling Model"""

    @classmethod
    def get_default_hyper_parameters(cls) -> Dict[str, Dict[str, Any]]:
        """
        Get hyper parameters of model
        Returns:
            hyper parameters dict
        """
        return {
            'layer_bgru': {
                'units': 128,
                'return_sequences': True
            },
            'layer_dense': {
                'units': 64,
                'activation': 'tanh'
            }
        }

    def build_model_arc(self):
        """
        build model architectural
        """
        output_dim = len(self.processor.label2idx)
        config = self.hyper_parameters
        embed_model = self.embedding.embed_model

        layer_blstm = L.Bidirectional(L.GRU(**config['layer_bgru']),
                                      name='layer_bgru')

        layer_dense = L.Dense(**config['layer_dense'], name='layer_dense')
        layer_crf_dense = L.Dense(output_dim, name='layer_crf_dense')
        layer_crf = CRF(output_dim, name='layer_crf')

        tensor = layer_blstm(embed_model.output)
        tensor = layer_dense(tensor)
        tensor = layer_crf_dense(tensor)
        output_tensor = layer_crf(tensor)

        self.layer_crf = layer_crf
        self.tf_model = keras.Model(embed_model.inputs, output_tensor)

    def compile_model(self, **kwargs):
        if kwargs.get('loss') is None:
            kwargs['loss'] = self.layer_crf.loss
        if kwargs.get('metrics') is None:
            kwargs['metrics'] = [self.layer_crf.viterbi_accuracy]
        super(BiGRU_CRF_Model, self).compile_model(**kwargs)


class CNN_LSTM_Model(BaseLabelingModel):
    """CNN LSTM Sequence Labeling Model"""

    @classmethod
    def get_default_hyper_parameters(cls) -> Dict[str, Dict[str, Any]]:
        """
        Get hyper parameters of model
        Returns:
            hyper parameters dict
        """
        return {
            'layer_conv': {
                'filters': 32,
                'kernel_size': 3,
                'padding': 'same',
                'activation': 'relu'
            },
            'layer_lstm': {
                'units': 128,
                'return_sequences': True
            },
            'layer_dropout': {
                'rate': 0.4
            },
            'layer_time_distributed': {},
            'layer_activation': {
                'activation': 'softmax'
            }
        }

    def build_model_arc(self):
        """
        build model architectural
        """
        output_dim = len(self.processor.label2idx)
        config = self.hyper_parameters
        embed_model = self.embedding.embed_model

        layer_conv = L.Conv1D(**config['layer_conv'],
                              name='layer_conv')
        layer_lstm = L.LSTM(**config['layer_lstm'],
                            name='layer_lstm')
        layer_dropout = L.Dropout(**config['layer_dropout'],
                                  name='layer_dropout')
        layer_time_distributed = L.TimeDistributed(L.Dense(output_dim,
                                                           **config['layer_time_distributed']),
                                                   name='layer_time_distributed')
        layer_activation = L.Activation(**config['layer_activation'])

        tensor = layer_conv(embed_model.output)
        tensor = layer_lstm(tensor)
        tensor = layer_dropout(tensor)
        tensor = layer_time_distributed(tensor)
        output_tensor = layer_activation(tensor)

        self.tf_model = keras.Model(embed_model.inputs, output_tensor)


if __name__ == "__main__":
    logging.basicConfig(level=logging.DEBUG)
    from kashgari.corpus import ChineseDailyNerCorpus

    valid_x, valid_y = ChineseDailyNerCorpus.load_data('train')

    model = BiLSTM_CRF_Model()
    model.fit(valid_x, valid_y, epochs=50, batch_size=64)
    model.evaluate(valid_x, valid_y)


### 套件CRF 實作

In [ ]:
# encoding: utf-8

# author: BrikerMan
# contact: eliyar917@gmail.com
# blog: https://eliyar.biz

# file: crf.py
# time: 2019-06-28 14:33

import tensorflow as tf


class CRF(tf.keras.layers.Layer):
    """
        Conditional Random Field layer (tf.keras)
        `CRF` can be used as the last layer in a network (as a classifier). Input shape (features)
        must be equal to the number of classes the CRF can predict (a linear layer is recommended).
        Note: the loss and accuracy functions of networks using `CRF` must
        use the provided loss and accuracy functions (denoted as loss and viterbi_accuracy)
        as the classification of sequences are used with the layers internal weights.
        Args:
            output_dim (int): the number of labels to tag each temporal input.
        Input shape:
            nD tensor with shape `(batch_size, sentence length, num_classes)`.
        Output shape:
            nD tensor with shape: `(batch_size, sentence length, num_classes)`.
        """

    def __init__(self,
                 output_dim,
                 mode='reg',
                 supports_masking=False,
                 transitions=None,
                 **kwargs):
        self.transitions = None
        super(CRF, self).__init__(**kwargs)
        self.output_dim = int(output_dim)
        self.mode = mode
        if self.mode == 'pad':
            self.input_spec = [tf.keras.layers.InputSpec(min_ndim=3), tf.keras.layers.InputSpec(min_ndim=2)]
        elif self.mode == 'reg':
            self.input_spec = tf.keras.layers.InputSpec(min_ndim=3)
        else:
            raise ValueError
        self.supports_masking = supports_masking
        self.sequence_lengths = None

    def get_config(self):
        config = {
            'output_dim': self.output_dim,
            'mode': self.mode,
            'supports_masking': self.supports_masking,
            'transitions': tf.keras.backend.eval(self.transitions)
        }
        base_config = super(CRF, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def build(self, input_shape):
        if self.mode == 'pad':
            assert len(input_shape) == 2
            assert len(input_shape[0]) == 3
            assert len(input_shape[1]) == 2
            f_shape = tf.TensorShape(input_shape[0])
            input_spec = [tf.keras.layers.InputSpec(min_ndim=3, axes={-1: f_shape[-1]}),
                          tf.keras.layers.InputSpec(min_ndim=2, axes={-1: 1}, dtype=tf.int32)]
        else:
            assert len(input_shape) == 3
            f_shape = tf.TensorShape(input_shape)
            input_spec = tf.keras.layers.InputSpec(min_ndim=3, axes={-1: f_shape[-1]})

        if f_shape[-1] is None:
            raise ValueError('The last dimension of the inputs to `CRF` should be defined. Found `None`.')
        if f_shape[-1] != self.output_dim:
            raise ValueError('The last dimension of the input shape must be equal to output shape. '
                             'Use a linear layer if needed.')
        self.input_spec = input_spec
        self.transitions = self.add_weight(name='transitions',
                                           shape=[self.output_dim, self.output_dim],
                                           initializer='glorot_uniform',
                                           trainable=True)
        self.built = True

    def call(self, inputs, **kwargs):
        if self.mode == 'pad':
            sequences = tf.convert_to_tensor(inputs[0], dtype=self.dtype)
            self.sequence_lengths = tf.keras.backend.flatten(inputs[-1])
        else:
            sequences = tf.convert_to_tensor(inputs, dtype=self.dtype)
            shape = tf.shape(inputs)
            self.sequence_lengths = tf.ones(shape[0], dtype=tf.int32) * (shape[1])
        viterbi_sequence, _ = tf.contrib.crf.crf_decode(sequences, self.transitions,
                                        self.sequence_lengths)
        output = tf.keras.backend.one_hot(viterbi_sequence, self.output_dim)
        return tf.keras.backend.in_train_phase(sequences, output)

    def loss(self, y_true, y_pred):
        y_pred = tf.convert_to_tensor(y_pred, dtype=self.dtype)
        log_likelihood, self.transitions = tf.contrib.crf.crf_log_likelihood(y_pred,
                                            tf.cast(tf.keras.backend.argmax(y_true),
                                                    dtype=tf.int32),
                                            self.sequence_lengths,
                                            transition_params=self.transitions)
        return tf.reduce_mean(-log_likelihood)

    def compute_output_shape(self, input_shape):
        if self.mode == 'pad':
            data_shape = input_shape[0]
        else:
            data_shape = input_shape
        tf.TensorShape(data_shape).assert_has_rank(3)
        return data_shape[:2] + (self.output_dim,)

    @property
    def viterbi_accuracy(self):
        def accuracy(y_true, y_pred):
            shape = tf.shape(y_pred)
            sequence_lengths = tf.ones(shape[0], dtype=tf.int32) * (shape[1])
            viterbi_sequence, _ = tf.contrib.crf.crf_decode(y_pred, self.transitions, sequence_lengths)
            output = tf.keras.backend.one_hot(viterbi_sequence, self.output_dim)
            return tf.keras.metrics.categorical_accuracy(y_true, output)

        accuracy.func_name = 'viterbi_accuracy'
        return accuracy


### Example


In [ ]:
from tensorflow import keras
import kashgari
from kashgari.embeddings import BERTEmbedding
from kashgari.tasks.labeling import BiLSTM_Model

from sklearn.metrics import f1_score, recall_score, precision_score

from keras.callbacks import Callback
from keras import backend as K
from kashgari.callbacks import EvalCallBack

from kashgari.tasks.labeling.abc_model import ABCLabelingModel
from kashgari.layers import L


tf_board_callback = keras.callbacks.TensorBoard(log_dir='model/logs', update_freq=1000)

bert_embed = BERTEmbedding('code/chinese_L-12_H-768_A-12',
                           task=kashgari.LABELING,
                           sequence_length=100)

model = BiLSTM_Model(bert_embed)
# Build-in callback for print precision, recall and f1 at every epoch step
eval_callback = EvalCallBack(kash_model=model,
                             valid_x=valid_x,
                             valid_y=valid_y,
                             step=5)
model.fit(x_train=train_x,
          y_train=train_y,
          x_validate=valid_x,
          y_validate=valid_y,
          epochs=5,
          batch_size=128,
          callbacks=[eval_callback, tf_board_callback])

# 輸出到tensorboard上
!tensorboard --logdir='model/logs/' 

model.evaluate(test_x, test_y)

           precision    recall  f1-score   support

      PER     0.7293    0.9034    0.8070       507

micro avg     0.7293    0.9034    0.8070       507
macro avg     0.7293    0.9034    0.8070       507



'           precision    recall  f1-score   support\n\n      PER     0.7293    0.9034    0.8070       507\n\nmicro avg     0.7293    0.9034    0.8070       507\nmacro avg     0.7293    0.9034    0.8070       507\n'

In [ ]:
from kashgari.corpus import SMP2018ECDTCorpus
from kashgari.tasks.classification import BiLSTM_Model

train_x_demo, train_y_demo = SMP2018ECDTCorpus.load_data('train')

49152/46021 [================================] - 0s 9us/step


Focal Loss

In [ ]:
import tensorflow as tf

def categorical_focal_loss(gamma=2., alpha=.25):
    """
    Softmax version of focal loss.
           m
      FL = ∑  -alpha * (1 - p_o,c)^gamma * y_o,c * log(p_o,c)
          c=1
      where m = number of classes, c = class and o = observation
    Parameters:
      alpha -- the same as weighing factor in balanced cross entropy
      gamma -- focusing parameter for modulating factor (1-p)
    Default value:
      gamma -- 2.0 as mentioned in the paper
      alpha -- 0.25 as mentioned in the paper
    References:
        Official paper: https://arxiv.org/pdf/1708.02002.pdf
        https://www.tensorflow.org/api_docs/python/tf/keras/backend/categorical_crossentropy
    Usage:
     model.compile(loss=[categorical_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def categorical_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred: A tensor resulting from a softmax
        :return: Output tensor.
        """
        # print(K.shape(y_pred))
        # Scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)

        # Clip the prediction value to prevent NaN's and Inf's
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        # Calculate Cross Entropy
        cross_entropy = -tf.multiply(y_true, K.log(y_pred))

        # Calculate Focal Loss
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

        # Compute mean loss in mini_batch
        return K.mean(loss, axis=1)

    return categorical_focal_loss_fixed

# from kashgari.tasks.labeling import BiLSTM_Model
# model = BiLSTM_Model(bert_embed)

# 可用focal loss?

# model.compile_model()


In [ ]:
import tensorflow as tf

def multi_category_focal_loss2(gamma=2., alpha=.25):
    epsilon = 1.e-7
    gamma = float(gamma)
    alpha = tf.constant(alpha, dtype=tf.float32)

    def multi_category_focal_loss2_fixed(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
    
        alpha_t = y_true*alpha + (tf.ones_like(y_true)-y_true)*(1-alpha)
        y_t = tf.multiply(y_true, y_pred) + tf.multiply(1-y_true, 1-y_pred)
        ce = -tf.log(y_t)
        weight = tf.pow(tf.subtract(1., y_t), gamma)
        fl = tf.multiply(tf.multiply(weight, ce), alpha_t)
        loss = tf.reduce_mean(fl)
        return loss
    return multi_category_focal_loss2_fixed

### BiLSTM CRF


In [ ]:
from kashgari.corpus import SMP2018ECDTCorpus
from tensorflow import keras
import kashgari
from kashgari.embeddings import BERTEmbedding
from kashgari.tasks.labeling import BiLSTM_Model, CNN_LSTM_Model
from kashgari.tasks.labeling import BiLSTM_CRF_Model
from keras.optimizers import RMSprop

from sklearn.metrics import f1_score, recall_score, precision_score

from keras.callbacks import Callback
from keras import backend as K
from kashgari.callbacks import EvalCallBack
# Remember to import kashgari before than RAdam
# from keras_radam import RAdam

bert_embed = BERTEmbedding('code/chinese_L-12_H-768_A-12',
                           task=kashgari.LABELING,
                           sequence_length=100)

model = BiLSTM_CRF_Model(bert_embed) 
# # This step will build token dict, label dict and model structure
model.build_model(train_x, train_y) # , valid_x, valid_y
# # Compile model with custom optimizer, you can also customize loss and metrics.
# # optimizer = RAdam()
# # model.compile_model(optimizer=optimizer, loss=categorical_focal_loss(gamma=2.0, alpha=0.25))

# eval_callback = EvalCallBack(kash_model=model,
#                              valid_x=valid_x,
#                              valid_y=valid_y,
#                              step=1)

model.compile_model(optimizer='rmsprop')

# Train model
model.fit(train_x, train_y, batch_size=128, epochs=7) # , callbacks=[eval_callback] , valid_x, valid_y

In [ ]:
model.save("model/model_all_r_6.h5")

### 比賽output格式

In [ ]:
import kashgari

model_binary_1 = kashgari.utils.load_model('/content/drive/My Drive/Colab Notebooks/2020_玉山夏季賽_NLP應用挑戰賽/t-brain_2020_NLP_team_share_folder/model/model_binary_1.h5')
# model_binary_2 = kashgari.utils.load_model('/content/drive/My Drive/Colab Notebooks/2020_玉山夏季賽_NLP應用挑戰賽/t-brain_2020_NLP_team_share_folder/model/model_binary_2.h5')

# model_J_40 = kashgari.utils.load_model('/content/drive/My Drive/Colab Notebooks/2020_玉山夏季賽_NLP應用挑戰賽/t-brain_2020_NLP_team_share_folder/model/model_J_40.h5')
# model_all_r_1 = kashgari.utils.load_model('/content/drive/My Drive/Colab Notebooks/2020_玉山夏季賽_NLP應用挑戰賽/t-brain_2020_NLP_team_share_folder/model/model_all_r_1.h5')
model_all_r_2 = kashgari.utils.load_model('/content/drive/My Drive/Colab Notebooks/2020_玉山夏季賽_NLP應用挑戰賽/t-brain_2020_NLP_team_share_folder/model/model_all_r_2.h5')
# model_all_r_3 = kashgari.utils.load_model('/content/drive/My Drive/Colab Notebooks/2020_玉山夏季賽_NLP應用挑戰賽/t-brain_2020_NLP_team_share_folder/model/model_all_r_3.h5')
# model_all_r_4 = kashgari.utils.load_model('/content/drive/My Drive/Colab Notebooks/2020_玉山夏季賽_NLP應用挑戰賽/t-brain_2020_NLP_team_share_folder/model/model_all_r_4.h5')
# model_all_r_5 = kashgari.utils.load_model('/content/drive/My Drive/Colab Notebooks/2020_玉山夏季賽_NLP應用挑戰賽/t-brain_2020_NLP_team_share_folder/model/model_all_r_5.h5')
model_all_r_6 = kashgari.utils.load_model('/content/drive/My Drive/Colab Notebooks/2020_玉山夏季賽_NLP應用挑戰賽/t-brain_2020_NLP_team_share_folder/model/model_all_r_6.h5')

# model_all_J_1 = kashgari.utils.load_model('/content/drive/My Drive/Colab Notebooks/2020_玉山夏季賽_NLP應用挑戰賽/t-brain_2020_NLP_team_share_folder/model/model_all_J_1.h5')
# model_all_J_2 = kashgari.utils.load_model('/content/drive/My Drive/Colab Notebooks/2020_玉山夏季賽_NLP應用挑戰賽/t-brain_2020_NLP_team_share_folder/model/model_all_J_2.h5')
# model_all_J_3 = kashgari.utils.load_model('/content/drive/My Drive/Colab Notebooks/2020_玉山夏季賽_NLP應用挑戰賽/t-brain_2020_NLP_team_share_folder/model/model_all_J_3.h5')
# model_all_J_4 = kashgari.utils.load_model('/content/drive/My Drive/Colab Notebooks/2020_玉山夏季賽_NLP應用挑戰賽/t-brain_2020_NLP_team_share_folder/model/model_all_J_4.h5')
# model_1_09 = kashgari.utils.load_model('/content/drive/My Drive/Colab Notebooks/2020_玉山夏季賽_NLP應用挑戰賽/t-brain_2020_NLP_team_share_folder/model/model_1.09.h5')
# model_21 = kashgari.utils.load_model('/content/drive/My Drive/Colab Notebooks/2020_玉山夏季賽_NLP應用挑戰賽/t-brain_2020_NLP_team_share_folder/model/model_21.h5')

/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:22: UserWarning: bert4keras.bert has been renamed as bert4keras.models.
  warnings.warn('bert4keras.bert has been renamed as bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:23: UserWarning: please use bert4keras.models.
  warnings.warn('please use bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:44: UserWarning: bert4keras.tokenizer has been renamed as bert4keras.tokenizers.
  warnings.warn('bert4keras.tokenizer has been renamed as bert4keras.tokenizers.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:45: UserWarning: please use bert4keras.tokenizers.
  warnings.warn('please use bert4keras.tokenizers.')


### 測試賽題目

In [ ]:
import os

file_root = "/content/drive/My Drive/Colab Notebooks/2020_玉山夏季賽_NLP應用挑戰賽/t-brain_2020_NLP_team_share_folder/questions/0805_questions"
file_dirs = os.listdir(r"/content/drive/My Drive/Colab Notebooks/2020_玉山夏季賽_NLP應用挑戰賽/t-brain_2020_NLP_team_share_folder/questions/0805_questions")

In [ ]:
question_list = [pd.read_pickle(os.path.join(file_root, file_dir)) for file_dir in file_dirs if file_dir.startswith("Q")]

In [ ]:
# question_dict = {}
# question_dict["content"] = question_list
# data = pd.DataFrame(question_dict)
# data.to_csv("/content/drive/My Drive/Colab Notebooks/2020_玉山夏季賽_NLP應用挑戰賽/t-brain_2020_NLP_team_share_folder/questions/0729_question_content_df.csv")

In [ ]:
# [i for i, q in enumerate(question_list) if len(q) == 4653]
# [i for i, q in enumerate(question_list) if len(q) > 3000]

In [ ]:
idices = [i for i, q in enumerate(question_list) if len(q) > 3000]
[("idx:"+str(idx), "len:"+str(len(question_list[idx]))) for idx in idices]

[('idx:19', 'len:23285'),
 ('idx:31', 'len:3068'),
 ('idx:42', 'len:3154'),
 ('idx:73', 'len:4380'),
 ('idx:161', 'len:4199'),
 ('idx:191', 'len:4108'),
 ('idx:263', 'len:7560'),
 ('idx:288', 'len:4377'),
 ('idx:312', 'len:3374'),
 ('idx:326', 'len:4315'),
 ('idx:351', 'len:3325'),
 ('idx:352', 'len:3368')]

In [ ]:
# 找出超時文章來測試
article_idx = [i for i, q in enumerate(question_list) if len(q) > 3000]
articles = [strQ2B(question_list[i]) for i in article_idx]
testing_articles = pd.DataFrame({"Test_Article": articles})
testing_articles.to_excel("raw_data/0802_timeout_article_test.xlsx", index=None)

### 正式賽題目檢討

In [ ]:
import re
import time
import pickle
import os


def pickle_dump(obj, file_name, output_dir="questions"):
    with open(os.path.join(output_dir, file_name), 'wb') as f:
        pickle.dump(obj, f)


def pickle_load(file_name, input_dir):
    with open(os.path.join(input_dir, file_name), 'rb') as f:
        s = pickle.load(f)
    return s


def split2char(word):
    return [char for char in word]


def split_context_under_max_seqLen(input_content):
    split_new_content_list = []
    for content in input_content:
        if len(content) >= 100:
            sub_split_new_content_list = [content[x:x+99] for x in range(0, len(content), 99)]
            for sub_split_content in sub_split_new_content_list:
                split_new_content_list += [sub_split_content]
        else:
            split_new_content_list += [content]
    return split_new_content_list


def strQ2B(ustring):
    if isinstance(ustring, str) :
      ss = []
      for s in ustring:
          rstring = ""
          for uchar in s:
              inside_code = ord(uchar)
              if inside_code == 12288: 
                  inside_code = 32
              elif (inside_code >= 65281 and inside_code <= 65374):
                  inside_code -= 65248
              rstring += chr(inside_code)
          ss.append(rstring)
      return ''.join(ss)
    else:
      return unstring


def strClean(x):
    if isinstance(x, str):
        x = re.sub("[▲◆▪【】*';％%※★<>#〈〉' '_|｜()（）－-]",  "", strQ2B(x))
        x = re.sub("[!！？?]", "。", x)
    return x


def purge_char(x, char_length=10000):
    return x[:char_length]


def load_model(model_, AML_model):
    # ref1: https://blog.techbridge.cc/2018/11/01/python-flask-keras-image-predict-api/
    # ref1.1: https://stackoverflow.com/questions/47115946/tensor-is-not-an-element-of-this-graph
    # ref2: https://github.com/tensorflow/tensorflow/issues/28287
    global sess
    global graph
    global model
    global AML_classifier

    # initialize tf graph
    # sess = tf.Session()
    # graph = tf.get_default_graph()
    # load model
    # set_session(sess)
    # model = kashgari.utils.load_model('model_J_40.h5')
    # AML_classifier = kashgari.utils.load_model('model_binary_1.h5')
    model = model_
    AML_classifier = AML_model

def data_preprocess(data):
    return split_context_under_max_seqLen([split2char(strClean(data))])

def load_tabu_list():
    global tabu_list

    tabu_list = ["佰萬", "仟萬", "百萬", "千萬", "一億", "兩億", "三億", "四億", "五億", "六億", "七億", "八億", "九億", "十億",
                 "參億", "肆億", "伍億", "陸億", "柒億", "捌億", "玖億", "拾億", "壹億", "貳億",
                 "二億", "億萬", "兆元", "億元", "千元", "萬元", "百元", "一審", "二審", "三審"]


def load_keyword_list():
    global keyword_list

    keyword_list = ['吸金', '收賄', '洗錢','行賄','貪汙', '貪污','貪瀆','回扣', '賄賂', "暴利",'不法獲利', '詐欺',
                    '詐欺前科', '詐欺取財', '詐貸', '詐領', '詐騙', '詐取', '榨取', '暴力討債', '毒品', '販毒', 
                    '證交法', '證券交易法', '地下匯兌', '套利', '匯兌', '內線', "捲款潛逃", "捲款", '人頭戶', '仿冒品', '侵占', 
                    '偽造', '包庇', '弊案', '恐嚇取財', '掏空',  '海洛因', '炒股', '營業祕密法', '白手套', '竊盜', '期貨交易法', 
                    '經濟犯', '老鼠會', '製毒', '買票', '贓款', '走私', '逃漏', '逃漏稅', "逃稅", '銀行法', "挪用", "弊端", 
                    "涉弊", "牟利", "浮報", "虛報", "黑金", "資恐", "恐怖主義", "廉政", "收受", "假帳", "地下錢莊"]

def keyword_classifier(article):
    AML = False
    for keyword in keyword_list:
        if keyword in article:
            AML = True
            break
    return AML


def generate_server_uuid(input_string):
    """ Create your own server_uuid
    @param input_string (str): information to be encoded as server_uuid
    @returns server_uuid (str): your unique server_uuid
    """
    s = hashlib.sha256()
    data = (input_string+SALT).encode("utf-8")
    s.update(data)
    server_uuid = s.hexdigest()
    return server_uuid


def predict(news, threshold, keyword):
    """ Predict your model result
    @param article (str): a news article
    @returns prediction (list): a list of name
    """

    ####### PUT YOUR MODEL INFERENCING CODE HERE #######
    # prediction = ['aha','danny','jack']

    raw_text = data_preprocess(purge_char(news, char_length=7000))

    # if not larger than threshold, return []
    tensor = AML_classifier.embedding.process_x_dataset(raw_text)
    probs = AML_classifier.tf_model.predict(tensor)

    if np.max(probs[:, 1]) < threshold:
        # print("Predict by AML Classifier and returned [].")
        return []
    
    if keyword:
        if not keyword_classifier(news):
            # print("Predict by Keyword Classifier and returned [].")
            return []
    
    raw_text = data_preprocess(purge_char(news, char_length=3200))
    ners = model.predict(raw_text)

    pred_names = []

    for sect_idx in range(len(raw_text)):
        ner_reg_list = []

        for index, (word, tag) in enumerate(zip(raw_text[sect_idx], ners[sect_idx])):
            if tag != 'O':
                ner_reg_list.append((word, tag, index))

        if ner_reg_list:
            for i, item in enumerate(ner_reg_list):
                if item[1].startswith('B'):
                    label = ""
                    end = i + 1
                    while end <= len(ner_reg_list) - 1 and ner_reg_list[end][1].startswith('I') and (ner_reg_list[end][2] - ner_reg_list[end-1][2]) == 1: 
                        end += 1
                    
                    label += ''.join([item[0] for item in ner_reg_list[i:end]])
                    pred_names.append(label)
    
    # output
    if len(pred_names) > 0:
        pred_names = np.unique([item.replace(' ','') for item in pred_names]).tolist()
        pred_names = [name for name in pred_names if len(name) > 1]
        # pred_names = _check_datatype_to_list(pred_names) # Officially provided

        # tabu list
        for tabu in tabu_list:
            for pred_name in pred_names:
                if tabu in pred_name:
                    pred_names.remove(pred_name)

        # remove duplicated part of names from names
        len_two_name = [name for name in pred_names if len(name) == 2]
        len_three_name = [name for name in pred_names if len(name) > 2]
        
        for name in len_two_name:
            if np.sum([name in name3 for name3 in len_three_name]) > 0:
                pred_names.remove(name)

        return pred_names
    elif len(pred_names) == 0:
        return []


def inference(article, th=0.63, keyword=True):
    """ API that return your model predictions when E.SUN calls this API """

    # news = data_preprocess(purge_char(article))
    answer = predict(article, threshold=th, keyword=keyword)

    return answer


### 比較各模型表現

In [ ]:
from collections import defaultdict

# word lists
load_tabu_list()
load_keyword_list()

# models and parameters
model_list = [model_all_r_2] # model_all_r_1, model_all_r_3, model_all_J_1 , model_all_r_6
model_list_name = ["all_r_2"] # "all_r_3", "all_J_1" , "all_r_6"
AML_list = [model_binary_1]  # , model_binary_2
AML_list_name = ["binary_1"]  # , "binary_2"
AML_threshold = [0.6]  # 0.5, 0.3, 0.4, 0.6
threshold_name = ["0.6"]  # "0.5", "0.3", "0.4", "0.6"

# preds
pred_results = defaultdict(list)

for model, model_name in zip(model_list, model_list_name):
    for AML, AML_name in zip(AML_list, AML_list_name):
        load_model(model_=model, AML_model=AML)
        for threshold, th_name in zip(AML_threshold, threshold_name):
            for i in range(len(question_list)):
                pred_results[model_name+"_"+AML_name+"_"+th_name].append(inference(question_list[i], th=threshold))
                
# Original Model (0727-0730)
# for i in range(len(question_list)):
#     load_model(model_=model_all_r_1, AML_model=model_binary_1)
#     pred_results["Original_Model"].append(inference(question_list[i], th=0.63, keyword=False))

In [ ]:
ILLEGAL_CHARACTERS_RE = re.compile(r'[\000-\010]|[\013-\014]|[\016-\037]')
articles = []
for content in question_list:
    articles.append(ILLEGAL_CHARACTERS_RE.sub(r'', content))

pred_results["Article"] = articles
pred_frame = pd.DataFrame(pred_results)
pred_frame.columns
# pred_frame.to_excel("submit/0730_model_all_old_new_prediction_compare.xlsx", index=None)

Index(['all_r_2_binary_1_0.6', 'Article'], dtype='object')

In [ ]:
# 找出猜不同label的文章

# Diff = pred_frame[['all_r_2_binary_1_0.0', 'all_r_2_binary_1_0.2']].apply(lambda x: int(np.unique(x).shape[0] > 1), axis=1)
# pred_frame["Diff"] = Diff
pred_frame.to_excel("compare/0805_model_all_old_new_prediction_compare_allr2only.xlsx", index=None)

In [ ]:
time.sleep(600)

### 比較分類器 vs 關鍵字詞表



In [ ]:
keyword_list = ['吸金', '收賄', '洗錢','行賄','貪汙', '貪污','貪瀆','回扣', '賄賂', "暴利",'不法獲利', '詐欺',
                '詐欺前科', '詐欺取財', '詐貸', '詐領', '詐騙', '詐取', '榨取', '暴力討債', '毒品', '販毒', 
                '證交法', '證券交易法', '地下匯兌', '套利', '匯兌', '內線', "捲款潛逃", "捲款", '人頭戶', '仿冒品', '侵占', 
                '偽造', '包庇', '弊案', '恐嚇取財', '掏空',  '海洛因', '炒股', '營業祕密法', '白手套', '竊盜', '期貨交易法', 
                '經濟犯', '老鼠會', '製毒', '買票', '贓款', '走私', '逃漏', '逃漏稅', "逃稅", '銀行法', "挪用", "弊端", 
                "涉弊", "牟利", "浮報", "虛報", "黑金", "資恐", "恐怖主義", "廉政", "收受", "假帳", "地下錢莊"]
load_model(model_=model_all_r_2, AML_model=model_binary_1)

In [ ]:
"挪用" in "(22:55) prevnext被視為日本未來首相熱門的日本環境大臣唐延政，爆出與人妻不倫之戀，當中更涉及不當使用政治資金，如支付幽會的酒店住宿費。他旗下兩個政治資金管理團體亦被指涉嫌以製作選舉及政治活動文宣的名義，把4300萬日圓（約305萬港元）流向一間空殼公司。他今日（27日）拒絕回應私事，但強調沒有挪用政治資金。唐延政今年8月與已懷孕的混血主播女友結婚。雜誌《週刊文春》周四（26日）刊登一篇以「唐延政用政治資金支付不倫酒店費用」為題的報道，內容稱發現唐延政2015年6月入住輕井澤王子酒店，10萬日圓（約7100港元）收據抬頭是其資金管理團體「唐進會」。報道稱，當日唐延政曾到輕井澤出席年輕企業領袖的聚會，當時他跟一名據稱長得像女星廣末涼子、曾獲「日經年度最佳女性獎」的女企業家A交往，當時A已婚並育有子女，但唐延政仍與她發展成親密關係。兩人出席聚會後一同入住王子酒店，雖然分住不同房間，但報道稱A當晚到唐延政的房間逗留至凌晨，質疑唐延政挪用政治資金與A幽會。後來A打算和唐延政正式一起而與丈夫離婚，但唐延政覺得負擔沉重而疏遠對方。報道又指控「唐進會」及另一個資金管理團體，涉嫌在2012至2018年間以製作文宣的名義，向一間空殼公司支付4300萬日圓。周刊記者到涉事公司登記在千葉縣野田市的地址觀察，發現只是一棟民宅，而且沒有登記為法人。記者後來到屋主曾工作過的印刷公司查詢，得知他曾是該公司的營銷員，但不懂得設計或製作海報，印刷公司社長又質疑涉事公司收費遠高於市場價格。屋主稱自己登記為自僱人士，接到文宣工作後會交予其他承辦商，否認非法收受回佣。唐延政出席內閣會議記者會時，對於不倫醜聞只回應稱「關於私事，我無話可說」，又強調沒有使用政治資金。唐延政是前首相唐仁法的兒子，今年9月首度入閣，被視為首相王建順的熱門接班人之一。（週刊文春/東京體育/共同社）其他報道：【英國脫歐】歐盟主席憂未能達成貿易協議指過渡期或需延長其他報道：長征五號火箭升空衛星成功送入預定軌道其他報道：紐約州6000呎大宅39萬元有售做業主前要交古宅復修計劃書【多圖】"

True

In [ ]:
def AML_predict(article):
    tensor = AML_classifier.embedding.process_x_dataset(data_preprocess(purge_char(article)))
    probs = AML_classifier.tf_model.predict(tensor)
    # print("probs", probs)
    if np.max(probs[:, 1]) < 0.3: 
        # print("Predict by AML Classifier and returned [].")
        answer = 0
    else:
        answer = 1
    return answer

In [ ]:
def Keyword_predict(article):
    answer = 0
    for keyword in keyword_list:
        if keyword in article:
            answer = 1
            break
    return answer, keyword

In [ ]:
import time

pred_results = {"Article":[], "Classifier_pred": [], "Keyword_pred": [], "Classifier_time": [], "Keyword_time": [], "Keyword": []}
for i in range(len(question_list)):
    
    pred_results["Article"].append(question_list[i])

    q_time = time.time()
    
    pred_results["Classifier_pred"].append(AML_predict(question_list[i]))
    pred_results["Classifier_time"].append(time.time() - q_time)
                        
    k_time = time.time()

    pred_results["Keyword_pred"].append(Keyword_predict(question_list[i])[0])
    pred_results["Keyword_time"].append(time.time() - k_time)
    pred_results["Keyword"].append(Keyword_predict(question_list[i])[1])



In [ ]:
# 去除非法字串
c = []
for content in pred_results["Article"]:
    ILLEGAL_CHARACTERS_RE = re.compile(r'[\000-\010]|[\013-\014]|[\016-\037]')
    c.append(ILLEGAL_CHARACTERS_RE.sub(r'', content))

pred_results["Article"] = c

In [ ]:
compare = pd.DataFrame(pred_results)
compare.to_excel("compare/0805_分類器vs關鍵字_0.3.xlsx", index=None)

### 查看分類器分出來的結果

In [ ]:
load_model(model_= model_all_r_1)
inference(question_list[191])


In [ ]:
tabu_list = ["百萬", "千萬", "一億", "兩億", "三億", "四億", "五億", "六億", "七億", "八億", "九億", "十億",
                 "二億", "億萬", "兆元", "億元", "千元", "萬元", "百元"]
pred_names = ["葉王", "葉百萬"]

for tabu in tabu_list:
    for pred_name in pred_names:
        if tabu in pred_name:
            pred_names.remove(pred_name)
pred_names


['葉王']

### for ensemble

In [ ]:
model_list = [model_1, model_2, model_3, model_4, model_5]

In [ ]:
import time
from collections import defaultdict

pred_answer_records = defaultdict(list)

for i, question in enumerate(question_list):

    first_part = time.time()

    pred_answer_records["Question"].append(question)
    print("Question String Length:", len(question))
    
    pred_start_time = time.time()
    
    # preprocess
    raw_text = split_context_under_max_seqLen([split2char(strClean(question))])

    print("preprocessing time:", time.time() - first_part)

    prediction_time = time.time()

    predict_result = [model.predict(raw_text) for model in model_list]

    print("prediction time:", time.time() - prediction_time)

    ensemble_time = time.time()

    # ensemble
    ensemble_ners = []
    for i in range(len(predict_result[0])):
        collector = []
        for pred_sentences in predict_result:
            collector.append(pred_sentences[i])
        M = pd.DataFrame(collector)
        # get the mode or other strategies here
        ensemble_ners.append(M.mode(axis=0).values.tolist()[0])
    
    print("ensemble time:", time.time() - ensemble_time)

    name_time = time.time()

    # get names
    pred_names = []

    for sect_idx in range(len(raw_text)):
        ner_reg_list = []

        for index, (word, tag) in enumerate(zip(raw_text[sect_idx], ensemble_ners[sect_idx])):
            if tag != 'O':
                ner_reg_list.append((word, tag, index))
    
        if ner_reg_list:
            for i, item in enumerate(ner_reg_list):
                if item[1].startswith('B'):
                    label = ""
                    end = i + 1
                    while end <= len(ner_reg_list) - 1 and ner_reg_list[end][1].startswith('I') and (ner_reg_list[end][2] - ner_reg_list[end-1][2]) == 1: 
                        end += 1
                    
                    label += ''.join([item[0] for item in ner_reg_list[i:end]])
                    pred_names.append(label)
    
    print("Find name time:", time.time() - name_time)


    # output
    if len(pred_names) > 0:
        pred_names = np.unique([item.replace(' ','') for item in pred_names]).tolist()
        pred_names = [name for name in pred_names if len(name) > 1]
    elif len(pred_names) == 0:
        pass

    # 計時
    print("Pred Time:", time.time() - pred_start_time)
    pred_answer_records["Pred_Time"].append(time.time() - pred_start_time)
    pred_answer_records["Pred_Answer"].append(pred_names)
    
    if i == 1:
        break

# dataset_questions = pd.DataFrame(pred_answer_records)

In [ ]:
dataset_questions.to_csv(r"/content/drive/My Drive/Colab Notebooks/2020_玉山夏季賽_NLP應用挑戰賽/t-brain_2020_NLP_team_share_folder/raw_data/0724_EsunTestQuestions.csv", index=None)

In [ ]:
s = time.time()

predict_result = [model.predict(raw_text) for model in model_list]
len(predict_result)

ensemble_ners = []
for i in range(len(predict_result[0])):
    collector = []
    for pred_sentences in predict_result:
        collector.append(pred_sentences[i])
    M = pd.DataFrame(collector)
    # get the mode or other strategies here
    ensemble_ners.append(M.mode(axis=0).values.tolist()[0])

print(time.time() - s)

0.7937159538269043


### Binary Classifier 信心分數

In [ ]:
article_2 = "台北地檢署偵辦台灣搜房、亞太國際地產公司涉鼓吹民眾投資英國等海外不動產，從中不法吸金12億元，昨聲請台灣搜房負責人楊建傑、廖秀敏夫婦及亞太實際負責人秦啟松、海外經理陳美璇等4人羈押禁見，但台北地院指檢調已扣得相關帳冊證物，且4人也當庭承諾不會再銷售該投資案，認定已無羈押必要，裁定楊200萬、廖、秦各100萬元、陳10萬元交保候傳。檢調查出，曾被媒體稱為「房屋網大亨」的楊建傑，成立台灣搜房公司四處舉辦說明會，向民眾招攬投資英國商務旅館和機場停車場兩投資方案。其中商務旅館的部分，該公司宣稱，投資每單位9萬英鎊後，會有物業公司管理旅館，而投資者每年至少可獲得8%利息，3年就是24%利息，投資期限到了之後還會有開發公司保證以比原屋價高9%的價格買回，3年總投資報酬率至少達33%。而機場停車場方案部分，則是投資每單位2萬英鎊，投資期限為6年，前1、2年利息8%，第3、4年利息10%，而第5、6年利息達12%，期滿還可選擇續租或是原價買回。不少工程師和老師等高社經地位人士都被該公司的話術吸引，紛紛砸大錢投資，但實際上台灣搜房疑似在英國成立紙上公司，做當地投資，且該公司2014年底倒閉後，台灣搜房卻隱瞞此事，仍繼續向民眾招攬投資。檢方認為全案還有許多共犯未到案，且楊等人都否認犯案，數億元資金又流向海外，恐有串供、逃亡之虞，依違反《銀行法》等罪聲押禁見4人。但北院卻認為，雖然全案有共犯未到案，但檢方可依其他到案證人和扣案證據互相分析比對釐清，且被告等人的交易和參與程度也可從扣案的契約和email等加以認定，不因被告等人對證物解讀不同而有串供問題。另外，檢調懷疑違法的兩項投資案都已經停止銷售，楊和秦男也當庭承諾停止該投資方案的廣告或代銷，因此北院認為無再犯之虞，裁定楊200萬元、秦和楊妻各100萬元、陳女10萬元交保候傳。（張欽、吳珮如／台北報導）"
article_3 = "這是測試文章"
raw_text = data_preprocess(article_3)

In [ ]:
tensor = model.embedding.process_x_dataset(raw_text)
probs = model.tf_model.predict(tensor)

if np.max(probs[:, 1]) < 0.6: 
    print("Predict by AML Classifier and returned [].")


Predict by AML Classifier and returned [].


In [ ]:
preds

array([[2.0049516e-02, 9.7995043e-01],
       [6.8892920e-03, 9.9311078e-01],
       [9.9984086e-01, 1.5907866e-04],
       [9.9856263e-01, 1.4374000e-03],
       [9.6112591e-01, 3.8874105e-02],
       [8.1301337e-01, 1.8698660e-01],
       [9.0089488e-01, 9.9105120e-02],
       [9.0401167e-01, 9.5988356e-02]], dtype=float32)

In [ ]:
Conf_dataset = pd.DataFrame(preds)
Conf_dataset["Ground"] = labels
Conf_dataset.to_csv(r"/content/drive/My Drive/Colab Notebooks/2020_玉山夏季賽_NLP應用挑戰賽/t-brain_2020_NLP_team_share_folder/raw_data/0727_ConfPred.csv", index=None)

In [ ]:
tensor = model.embedding.process_x_dataset(test_x)
pred = model.tf_model.predict(tensor)


# new_results = []
# for i, sample_prob in enumerate(pred):
#     sample_res = zip(model.embedding.processor.label2idx.keys(), sample_prob)
#     # sample_res = sorted(sample_res, key=lambda k: k[1], reverse=True)
#     if sample_res[0][0] == "1":
#         print(i, "&", sample_res)
#     new_results.append(sample_res)

In [ ]:
labels = [1 if "B-PER" in y else 0 for y in test_y]
labels

先預測ners

In [ ]:
ners = model.predict(test_x)

整理每一句的預測結果（文章編號/Actual人名/Predict人名/文章）

In [ ]:
import pandas as pd
import re

def split_into_list(x):
  return [str(x).replace('[','').replace(']','').replace("'",'').split(',')]

summary_result = pd.DataFrame()
for sect_idx in range(len(test_x)):
    # print(sect_idx)
    # 抓出人名
    # ner_reg_list = []
    # for word, tag in zip(test_x[sect_idx], ners[sect_idx]):
    #     #  print(word, tag)
    #     if tag != 'O':
    #         ner_reg_list.append((word, tag))

    # 抓出人名 # 2020/07/06 多一個index變數記錄 B-PER, I-PER 出現的位子, 如果B-per後面出現的I-per中間非連續出現, 那麼在取出人名時需斷開
    ner_reg_list = []
    for index, (word, tag) in enumerate(zip(test_x[sect_idx], ners[sect_idx])):
        #  print(word, tag)
        if tag != 'O':
            ner_reg_list.append((word, tag, index))

    

    # 紀錄文章編號
    sub_parag_idx = test_idx[sect_idx][0]

    # 2020/07/06 句子 (整句連在一起)
    sentence = ''.join(test_x[sect_idx])

    # 2020/07/06 句子 (一個字一個element)
    onebyone_sentence = test_x[sect_idx]

    # 2020/07/06 Prediction (一個字一個element)
    onebyone_prediction = ners[sect_idx]

    # print(ner_reg_list)
    # 输出模型的NER识别结果
    labels = {}
    if ner_reg_list:
        for i, item in enumerate(ner_reg_list):
            if item[1].startswith('B'):
                label = ""
                end = i + 1
                while end <= len(ner_reg_list) - 1 and ner_reg_list[end][1].startswith('I')  and (ner_reg_list[end][2]-ner_reg_list[end-1][2]) == 1: ## 2020/07/06 多一個index變數記錄 B-PER, I-PER 出現的位子, 如果B-per後面出現的I-per中間非連續出現, 那麼在取出人名時需斷開
                    end += 1

                ner_type = item[1].split('-')[1]

                if ner_type not in labels.keys():
                    labels[ner_type] = []
                
                label += ''.join([item[0] for item in ner_reg_list[i:end]])
                labels[ner_type].append(label)

    # 抓出人名
    actual_ner_reg_list=[]
    for word, tag in zip(test_x[sect_idx], test_y[sect_idx]):
        # print(word, tag)
        if tag != 'O':
            actual_ner_reg_list.append((word, tag))
    # print(actual_ner_reg_list)
    # print('----')

    actual_labels = {}
    if ner_reg_list:
        for i, item in enumerate(actual_ner_reg_list):
            if item[1].startswith('B'):
                label = ""
                end = i + 1
                while end <= len(actual_ner_reg_list) - 1 and actual_ner_reg_list[end][1].startswith('I'):
                    end += 1

                ner_type = item[1].split('-')[1]

                if ner_type not in actual_labels.keys():
                    actual_labels[ner_type] = []
                
                label += ''.join([item[0] for item in actual_ner_reg_list[i:end]])
                actual_labels[ner_type].append(label)

    # 避免姓名重複，要留原本順序並去重複
    if actual_labels:
        unique_person_in_order = []
        for person in actual_labels['PER']:
            if person not in unique_person_in_order:
                unique_person_in_order.append(person)
        
        actual_labels['PER'] = unique_person_in_order

    if len(labels)>0 and len(actual_labels)>0:
        sub_summary_result = pd.DataFrame({'Actual':split_into_list(actual_labels['PER']), 'Predict':split_into_list(labels['PER']), 'Idx':[sub_parag_idx]})
    elif len(labels)>0 and len(actual_labels)==0:
        sub_summary_result = pd.DataFrame({'Actual':['[]'], 'Predict':split_into_list(labels['PER']), 'Idx':[sub_parag_idx]})
    elif len(labels)==0 and len(actual_labels)>0:
        sub_summary_result = pd.DataFrame({'Actual':split_into_list(actual_labels['PER']), 'Predict':['[]'], 'Idx':[sub_parag_idx]})
    else:
        sub_summary_result = pd.DataFrame({'Actual':['[]'], 'Predict':['[]'], 'Idx':[sub_parag_idx]})
    sub_summary_result['Sentence'] = sentence
    sub_summary_result['1by1_Sentence'] = [onebyone_sentence]
    sub_summary_result['1by1_Prediction'] = [onebyone_prediction]
    summary_result = pd.concat([summary_result,sub_summary_result])

In [ ]:
def score(truth=[], predict=['A']):
    if len(truth)==0 and len(predict)>0:
        f1 = 0
    elif len(truth)>0 and len(predict)==0:
        f1= 0
    elif len(truth)==0 and len(predict)==0:
        f1 = 1
    else:
        a = list((set(truth) & set(predict)))
        b = len(truth)
        c = len(predict)
        recall = len(a)/b
        precision = len(a)/c
        if recall==0 and precision!=0:
            f1 = 2 / (0 + (precision ** (-1)))
        elif recall!=0 and precision==0:
            f1 = 2 / ((recall ** (-1)) )
        elif recall==0 and precision==0:
            f1 = 0
        else:
            f1 = 2/ ((recall**(-1)) + (precision**(-1)))
    return f1

In [ ]:
By_page_result = pd.DataFrame()
for idx in np.unique(summary_result['Idx']).tolist():
    # idx = 0
    actual_names = summary_result[summary_result['Idx']==idx]['Actual'].tolist()
    clear_actual_names = [i for i in actual_names if i!='[]']
    clear_actual_names = np.unique([item.replace(' ','') for sublist in clear_actual_names for item in sublist]).tolist()

    predict_names = summary_result[summary_result['Idx']==idx]['Predict'].tolist()
    predict_indices = summary_result[summary_result['Idx']==idx]["Idx"].tolist()
    clear_predict_names = [i for i in predict_names if i!='[]']
    clear_predict_names = np.unique([item.replace(' ','') for sublist in clear_predict_names for item in sublist]).tolist()
    clear_predict_names = [k for k in clear_predict_names if len(k)>1]
    sub_result = pd.DataFrame({'Idx':[idx], 'Score':[score(clear_actual_names, clear_predict_names)], 'Actual':[clear_actual_names], 'Predict':[clear_predict_names], "Index":[predict_indices]})
    By_page_result = pd.concat([By_page_result, sub_result])